# 如何计算时间滑动窗口统计变量

In [ ]:
import pandas as pd
import numpy as np
import datetime
import time
import timeit

In [95]:
df = pd.DataFrame(np.random.randn(10000,10),index = xrange(10000),columns = list('abcdefghij'))

In [96]:
df

,a,b,c,d,e,f,g,h,i,j
0,0.334908,1.772868,-0.923213,1.052904,1.524496,-0.139358,-1.526997,0.233452,-0.904989,-0.364495
1,0.766120,0.258490,-0.187979,1.528881,1.671347,-1.764962,-2.531285,0.642724,0.134457,-0.155350
2,0.109673,1.061700,-0.545651,-0.516764,1.483462,0.686994,1.428703,-2.175427,0.704613,0.074799
3,-1.036246,1.457020,0.667350,-1.875718,1.403239,-0.584255,-1.905242,-0.763530,-0.107767,1.074940
4,-0.285157,0.243611,0.132073,-0.003974,1.093359,-1.419841,0.854478,-1.522734,-0.954373,0.100930
5,1.406196,-0.057472,0.231067,0.441137,0.744521,-0.007519,2.112322,0.441744,-0.375529,0.346551
6,-0.441358,1.646965,1.359210,-1.072371,0.227582,-0.381263,0.672033,-0.935970,-1.053246,-0.155131
7,-1.045184,1.013175,1.492118,-0.155969,1.690675,-1.698750,0.767843,0.893243,-0.387118,-0.194366
8,0.216430,-0.154969,-0.383590,0.096926,-0.252922,0.297951,-0.098447,-0.712357,-1.411124,0.292024
9,-0.166080,-0.819054,-0.393750,3.400256,-1.503749,0.010735,1.166898,-0.149234,0.094313,0.037064


In [97]:
x = datetime.date.today()

In [98]:
x

datetime.date(2016, 7, 11)

In [99]:
rng = pd.date_range(x,periods = df.shape[0],freq='D')

In [100]:
rng

DatetimeIndex(['2016-07-11', '2016-07-12', '2016-07-13', '2016-07-14',
               '2016-07-15', '2016-07-16', '2016-07-17', '2016-07-18',
               '2016-07-19', '2016-07-20',
               ...
               '2043-11-17', '2043-11-18', '2043-11-19', '2043-11-20',
               '2043-11-21', '2043-11-22', '2043-11-23', '2043-11-24',
               '2043-11-25', '2043-11-26'],
              dtype='datetime64[ns]', length=10000, freq='D')

In [101]:
df = df.set_index(rng)

In [102]:
df

,a,b,c,d,e,f,g,h,i,j
2016-07-11,0.334908,1.772868,-0.923213,1.052904,1.524496,-0.139358,-1.526997,0.233452,-0.904989,-0.364495
2016-07-12,0.766120,0.258490,-0.187979,1.528881,1.671347,-1.764962,-2.531285,0.642724,0.134457,-0.155350
2016-07-13,0.109673,1.061700,-0.545651,-0.516764,1.483462,0.686994,1.428703,-2.175427,0.704613,0.074799
2016-07-14,-1.036246,1.457020,0.667350,-1.875718,1.403239,-0.584255,-1.905242,-0.763530,-0.107767,1.074940
2016-07-15,-0.285157,0.243611,0.132073,-0.003974,1.093359,-1.419841,0.854478,-1.522734,-0.954373,0.100930
2016-07-16,1.406196,-0.057472,0.231067,0.441137,0.744521,-0.007519,2.112322,0.441744,-0.375529,0.346551
2016-07-17,-0.441358,1.646965,1.359210,-1.072371,0.227582,-0.381263,0.672033,-0.935970,-1.053246,-0.155131
2016-07-18,-1.045184,1.013175,1.492118,-0.155969,1.690675,-1.698750,0.767843,0.893243,-0.387118,-0.194366
2016-07-19,0.216430,-0.154969,-0.383590,0.096926,-0.252922,0.297951,-0.098447,-0.712357,-1.411124,0.292024
2016-07-20,-0.166080,-0.819054,-0.393750,3.400256,-1.503749,0.010735,1.166898,-0.149234,0.094313,0.037064


In [103]:
arr = df.values

In [104]:
arr

array([[ 0.33490815,  1.7728678 , -0.92321311, ...,  0.23345243,
        -0.90498938, -0.36449469],
       [ 0.76612027,  0.25848961, -0.18797914, ...,  0.64272357,
         0.13445674, -0.15535012],
       [ 0.10967294,  1.06170044, -0.54565147, ..., -2.1754273 ,
         0.70461324,  0.07479865],
       ..., 
       [-1.10994491,  0.13485614, -1.55195582, ..., -1.5929953 ,
         1.19570118, -0.20196886],
       [ 2.38324604,  0.39459914, -0.90810632, ...,  0.44286329,
        -0.44380158,  0.60871314],
       [-1.32991394,  0.15954519, -0.86354692, ...,  0.56525075,
        -1.02018577, -0.97845277]])

In [109]:
t1 = time.clock()
%timeit df['avg_10'] = pd.rolling_mean(df.a,10)
t2 = time.clock()

1000 loops, best of 3: 562 µs per loop


In [110]:
df

,a,b,c,d,e,f,g,h,i,j,avg_10
2016-07-11,0.334908,1.772868,-0.923213,1.052904,1.524496,-0.139358,-1.526997,0.233452,-0.904989,-0.364495,NaN
2016-07-12,0.766120,0.258490,-0.187979,1.528881,1.671347,-1.764962,-2.531285,0.642724,0.134457,-0.155350,NaN
2016-07-13,0.109673,1.061700,-0.545651,-0.516764,1.483462,0.686994,1.428703,-2.175427,0.704613,0.074799,NaN
2016-07-14,-1.036246,1.457020,0.667350,-1.875718,1.403239,-0.584255,-1.905242,-0.763530,-0.107767,1.074940,NaN
2016-07-15,-0.285157,0.243611,0.132073,-0.003974,1.093359,-1.419841,0.854478,-1.522734,-0.954373,0.100930,NaN
2016-07-16,1.406196,-0.057472,0.231067,0.441137,0.744521,-0.007519,2.112322,0.441744,-0.375529,0.346551,NaN
2016-07-17,-0.441358,1.646965,1.359210,-1.072371,0.227582,-0.381263,0.672033,-0.935970,-1.053246,-0.155131,NaN
2016-07-18,-1.045184,1.013175,1.492118,-0.155969,1.690675,-1.698750,0.767843,0.893243,-0.387118,-0.194366,NaN
2016-07-19,0.216430,-0.154969,-0.383590,0.096926,-0.252922,0.297951,-0.098447,-0.712357,-1.411124,0.292024,NaN
2016-07-20,-0.166080,-0.819054,-0.393750,3.400256,-1.503749,0.010735,1.166898,-0.149234,0.094313,0.037064,-0.014070


In [111]:
#numpy methods
#def f(arr,td):
 #   x = np.zeros(arr.shape)
#    for i in xrange(td):
        

In [113]:
t3 = time.clock()
%timeit y = np.convolve(df.values[:,0],np.ones(10,),'valid')/10
t4 = time.clock()

The slowest run took 14.19 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 85.8 µs per loop


In [76]:
arr[:,0]

array([ 0.60329448,  0.03882076,  0.27066905, ...,  0.39340854,
       -1.07165464, -0.36606199])

In [77]:
y

array([-0.01030826, -0.12069752, -0.099219  , ..., -0.2750428 ,
       -0.49234789, -0.54564441])

In [78]:
y.shape

(9991L,)